In [15]:
import src.load_data as load_data
import tensorflow as tf
import numpy as np

## Loading data

In [3]:
x_train, y_train, x_test, y_test = load_data.load_cifar10()

170508288/170498071 [==============================] - 81s 0us/step


In [22]:
x_train.shape

(50000, 32, 32, 3)

In [24]:
IMG_SHAPE = x_train.shape[1:]
BATCH_SIZE = 32

In [9]:
# Load the VGG16 model
base_model = tf.keras.applications.vgg16.VGG16(
    include_top=False, weights="imagenet", classifier_activation="softmax", input_shape=IMG_SHAPE,
)
base_model.trainable = False

In [19]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(10) # 10 classes

In [20]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = inputs
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [21]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [27]:
model.fit(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=2,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    #validation_data=(x_test, y_test),
)

Epoch 1/2
  1/313 [..............................] - ETA: 1:57 - loss: 3.3766 - accuracy: 0.0625

KeyboardInterrupt: 

In [10]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

### Creating the pipeline
- Preprocess images
- 